<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_online_0616/blob/main/tibame_night_20250708.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


```
HTTP代碼
1. 2: 成功
2. 3: 轉址
3. 4: 錯誤
403 Forbidden
1. IP被ban: 換ip/等
2. 模仿不夠像: 缺少必要headers
```





```
如果你的html區塊是沒有class/id可以讓你篩選的
1. 找他老爸(必須有class/id)
2. 自己想辦法寫一個篩選方式(文字/超連結網址)

```



In [1]:
import urllib.request as req

# 相對路徑:
# a.txt: 現在這個目錄下的a.txt
# ./a.txt: 現在(.)這個目錄下的a.txt
# ../a.txt: 前一層目錄(..)的a.txt
def get_imgur_image(url, dirname="."):
    # dirname = "file"
    # url = "https://i.imgur.com/OSwXeuz.jpeg"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")
    resp = req.urlopen(r)
    content = resp.read()

    # 檔案讀寫
    # 1. 純文字: r/w  encoding="utf-8"
    # 2. 非純文字(jpeg, png, docx, xlsx, pdf...): rb/wb
    fp = dirname + "/" + url.split("/")[-1]
    print("存放位置:", fp)
    with open(fp, "wb") as f:
        f.write(content)

# url = "https://i.imgur.com/H8i2ZVT.jpeg"
# get_imgur_image(url, "test")

In [6]:
import json
import os
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1751797464.A.265.html"
# 儲存的路徑
dirname = url.split("/")[-1]
if not os.path.exists(dirname):
    os.makedirs(dirname)

r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")
resp = req.urlopen(r)
content = resp.read()
html = bs.BeautifulSoup(content)
# find/find_all
# .text .get_text() ["href"]

# 作者/標題/時間/看板
metas = html.find_all("span", {"class":"article-meta-value"})
author = metas[0]
board = metas[1]
title = metas[2]
post_time = metas[3]
pushes = html.find_all("div", {"class":"push"})


author_text = author.text
board_text = board.text
title_text = title.text
post_time_text = post_time.text

push_list = []
for p in pushes:
    push_meta = p.find_all("span")
    push_trans = {"推":1, "噓":-1, "→":0}
    push_data = {
        "type":push_trans[push_meta[0].text.strip()],
        "uid":push_meta[1].text.strip(),
        "text":push_meta[2].text.strip().replace(": ", ""),
        "ipdatetime":push_meta[3].text.strip()
    }
    push_list.append(push_data)

# 下載所有圖片
allow_sub = ["jpg", "jpeg", "png", "gif"]
links = html.find_all("a")
for l in links:
    href = l["href"]
    sub = href.split(".")[-1]
    if sub.lower() in allow_sub:
        print("符合格式, 下載:", href)
        get_imgur_image(href)

# 內文: 沒有任何區塊把內文包起來, 先找到大區塊, 把區塊裡面你不要的東西去掉(extract)
main_content = html.find("div", {"id":"main-content"})
for e in html.find_all("div", {"class":"article-metaline"}):
    e.extract()
for e in html.find_all("div", {"class":"article-metaline-right"}):
    e.extract()
for e in html.find_all("div", {"class":"push"}):
    e.extract()
# print(main_content)
content_text = main_content.text.strip()


row = {
    "author":author_text,
    "title":title_text,
    "post_time":post_time_text,
    "board":board_text,
    "pushes":push_list,
    "content":content_text,
}
# print(row)
fn = url.split("/")[-1] + ".json"
with open(fn, "w", encoding="utf-8") as f:
    json.dump(row, f, indent=4, ensure_ascii=False)

符合格式, 下載: https://i.imgur.com/OSwXeuz.jpeg
存放位置: ./OSwXeuz.jpeg
符合格式, 下載: https://i.imgur.com/c2CmTdy.jpeg
存放位置: ./c2CmTdy.jpeg
符合格式, 下載: https://i.imgur.com/WDzTRGh.jpeg
存放位置: ./WDzTRGh.jpeg
符合格式, 下載: https://i.imgur.com/Ue2jqqf.jpeg
存放位置: ./Ue2jqqf.jpeg
符合格式, 下載: https://i.imgur.com/ejEwGXA.jpeg
存放位置: ./ejEwGXA.jpeg
符合格式, 下載: https://i.imgur.com/H8i2ZVT.jpeg
存放位置: ./H8i2ZVT.jpeg
符合格式, 下載: https://i.imgur.com/Hhc8hwu.gif
存放位置: ./Hhc8hwu.gif


In [2]:
f = open("a.txt", "w", encoding="utf-8")
f.write("abc")
f.close()

with open("a.txt", "w", encoding="utf-8") as f:
    f.write("abc")

In [3]:
import os

dirname = "a/b/c/d"
if not os.path.exists(dirname):
    os.makedirs(dirname)

In [5]:
# 資料夾路徑如果是用字串的相加其實很危險
# 萬一沒打斜線/或者打了兩個斜線
os.path.join("a/", "b")
os.path.join("a", "b/", "c", "d")

'a/b/c/d'

In [ ]:
# 解釋os.path/requests